<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2024-1/Insumos/BonoM_Cup%C3%B3nCero_practica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 63 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [ ]:
%%R
#Tasa alambrada
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

In [ ]:
%%R
btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"
tfcupon=cbind( 0.065,	0.0675,	0.07,	0.075,	0.078) #Tasafija del cupón
plazos=cbind( 378,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon=cbind( 182,	182, 182, 182, 182) #plazos fijos de cada cupón
contratos=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal=100
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

In [ ]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos)


      N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
      VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
      contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
      nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
      plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
      tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
      ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


      plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales

      for (j in (1:m))
      {
        if (j==1)
        {
          VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
          contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
          plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
          tasafijaT[,1:sum(N[1:j])]=seq(tfcupon[j],tfcupon[j])
          ulNomT[,sum(N[1:j])]=contratos[j]
        }
        else
        {
          VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
          contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])
          plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
          tasafijaT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tfcupon[j],tfcupon[j])
          ulNomT[,sum(N[1:j])]=contratos[j]
        }
      }

      Xvp=matrix(0,n,ncol(VTplazos))

      for (i in (1:n))
      {
        Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}

      }

      print(VTplazos)
      ulNomT
      print(tasafijaT)
      N

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   14  196  378   41  223  405    4  186  368   550   108   290   472   654
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   836  1018  1200   162   344   526   708   890  1072  1254  1436  1618
     [,27]
[1,]  1800
      [,1]  [,2]  [,3]   [,4]   [,5]   [,6] [,7] [,8] [,9] [,10] [,11] [,12]
[1,] 0.065 0.065 0.065 0.0675 0.0675 0.0675 0.07 0.07 0.07  0.07 0.075 0.075
     [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24]
[1,] 0.075 0.075 0.075 0.075 0.075 0.078 0.078 0.078 0.078 0.078 0.078 0.078
     [,25] [,26] [,27]
[1,] 0.078 0.078 0.078
[1]  3  3  4  7 10


$$ V_{cc}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$

Donde
$V_{cc}$: Valor del Bono bajo tasa de curva cupón cero (Precio Sucio)
$N$: Valor Nominal del bono.
$C$: Número de contratos.
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.
$p_{i}$: Plazo acumulado (en días) al cupón $i$.
$t_{c}$: Tasa cupón fija
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos cupón cero según el plazo acumulado al pago del cupón $i$
Se tiene nun factor de riesgo subyacente (curva gubernamental) y $n$ factores de riesgos.


In [ ]:
%%R
library(plyr)
bonoMcccero=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, N)
{
        V0=matrix(0,1,nrow(as.matrix(N)))
          V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+Xvp*VTplazos/360)))*nominal
        for (j in (1:nrow(as.matrix(N))))
        {
          if(j==1)
          {
            V0[j]=sum(V0f[j:N[j]])
          }
          else
          {
            V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
          }
        }
        V0
}


V0=bonoMcccero(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], N)
V0

        [,1]     [,2]    [,3]     [,4]    [,5]
[1,] 2236455 -2942426 2969778 -4643930 1009445


##TRADUCCIÓN A PYTHON

In [ ]:
#celda 1

In [ ]:
#celda 2

In [ ]:
#celda 3

# Ingresar la ecuación al chat (que lo haga para valuar varios bonos en una función):
$$ V_{cc}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$

Donde
$V_{cc}$: Valor del Bono bajo tasa de curva cupón cero (Precio Sucio)\
$N$: Valor Nominal del bono.\
$C$: Número de contratos.\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c}$: Tasa cupón fija\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos cupón cero según el plazo acumulado al pago del cupón $i$\
Se tiene nun factor de riesgo subyacente (curva gubernamental) y $n$ factores de riesgos.
